In [2]:
#load libraries

import numpy as np
import pandas as pd
from sklearn.metrics import \
    r2_score, get_scorer
from sklearn.linear_model import \
    Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import \
    StandardScaler, PolynomialFeatures
from sklearn.model_selection import \
    KFold, RepeatedKFold, GridSearchCV, \
    cross_validate, train_test_split


from sklearn.preprocessing import MinMaxScaler

from scipy.stats.mstats import winsorize

from tqdm import tqdm

In [3]:
#load data --> winsorize data --> scale data

df = pd.read_csv ('/Volumes/USB THOMAS/THESIS NEW/DATA/8. MERGE FILES (FINAL DATASET)/Dataset_unscaled.csv', index_col=0)
df.sort_values(by=['year','month'], inplace=True)
df = df.reset_index(drop=True)

df.iloc[:,16:] = df.iloc[:,16:].apply(lambda x : winsorize(x,(0.1,0.1)))

cols_to_scale = list(df.iloc[:,15:].columns)
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

df[cols_to_scale] = min_max_scaler.fit_transform(df[cols_to_scale])

In [4]:
R2_list_lm = []

for name in df.iloc[:,9:15]:
    
    df_iter = df[df[name] <= df[name].quantile(0.95)].copy()
    
    for p in range(1,6):
    
        years = np.arange(1996, 2013+p, 1)

        locals()['df_train_' + str(p)] = df_iter[df_iter.year.isin(years)]

        year = [2013 + p,2014 + p,2015 + p]

        locals()['df_val_' + str(p)] = df_iter[df_iter.year.isin(year)]

        year = [2016 + p]

        locals()['df_test_' + str(p)] = df_iter[df_iter.year.isin(year)]

    true = []
    predicted = []

    for j in tqdm(range(1,6)):
        y_train = globals()['df_train_'+str(j)][name]
        y_val = globals()['df_val_'+str(j)][name]
        y_test = globals()['df_test_'+str(j)][name]

        X_train = globals()['df_train_'+str(j)].iloc[:,16:]
        X_val = globals()['df_val_'+str(j)].iloc[:,16:]
        X_test = globals()['df_test_'+str(j)].iloc[:,16:]

        model = LinearRegression()

        model.fit(X_train,y_train)

        y_pred_test = model.predict(X_test)

        true.extend(y_test)

        predicted.extend(y_pred_test)
    
    R2_list_lm.append([name,r2_score(true, predicted)*100])
    
print(R2_list_lm)

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  9.09it/s]

[['IV_D_C_30', 9.02490778628927], ['IV_D_P_30', 9.483842328430626], ['IV_D_C_60', 11.35495927399528], ['IV_D_P_60', 11.72974882399811], ['IV_D_C_91', 11.472049914495653], ['IV_D_P_91', 10.940698664605952]]


In [6]:
# FINVARS ONLY

R2_list_lm = []

for name in df.iloc[:,9:10]:
    
    df_iter = df[df[name] <= df[name].quantile(0.95)].copy()
    
    for p in range(1,6):
    
        years = np.arange(1996, 2013+p, 1)

        locals()['df_train_' + str(p)] = df_iter[df_iter.year.isin(years)]

        year = [2013 + p,2014 + p,2015 + p]

        locals()['df_val_' + str(p)] = df_iter[df_iter.year.isin(year)]

        year = [2016 + p]

        locals()['df_test_' + str(p)] = df_iter[df_iter.year.isin(year)]

    true = []
    predicted = []

    for j in tqdm(range(1,6)):
        y_train = globals()['df_train_'+str(j)][name]
        y_val = globals()['df_val_'+str(j)][name]
        y_test = globals()['df_test_'+str(j)][name]

        X_train = globals()['df_train_'+str(j)].iloc[:,16:60]
        X_val = globals()['df_val_'+str(j)].iloc[:,16:60]
        X_test = globals()['df_test_'+str(j)].iloc[:,16:60]

        model = LinearRegression()

        model.fit(X_train,y_train)

        y_pred_test = model.predict(X_test)

        true.extend(y_test)

        predicted.extend(y_pred_test)
    
    R2_list_lm.append([name,r2_score(true, predicted)*100])
    
print(R2_list_lm)

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  9.66it/s]

[['IV_D_C_30', 8.940095791380177]]
